In [172]:
import os,sys
sys.path.append("S:/mo8557/ats/repos/ats-dev/tools/utils")

import matplotlib
%matplotlib

#print(sys.path)

import numpy as np
from matplotlib import pyplot as plt
import h5py

import parse_ats 
import transect_data
import mesh

# Can't be a mesh where the test has been cut off in the middle
mesh_dir="./test7/run21_19Feb18_Static_Parameters"



Using matplotlib backend: Qt5Agg


In [173]:
# run something on your mesh
#
# - must be the same mesh
# - must be the same partitioning
#  (mpiexec -n YY meshconvert --partition --partition-method=2 --classify ...)
# - must be the same stratigraphy

In [174]:
xyz = mesh.meshElemCentroids(directory=mesh_dir)
# Calls an ats utility called 'meshElemCentroids' (in ./tools/utils), which finds the centroid of
# elements in an already created mesh.  The utility calls mesh name 'visdump_mesh.h5', which is the 
# the default name for a mesh output created by ATS.  Specify its location in the directory

print xyz


[[  5.00000000e+00   5.00000000e-01   4.00900000e+01]
 [  5.00000000e+00   5.00000000e-01   4.00700000e+01]
 [  5.00000000e+00   5.00000000e-01   4.00500000e+01]
 ..., 
 [  5.95000000e+02   5.00000000e-01   6.84841358e+01]
 [  5.95000000e+02   5.00000000e-01   5.95404167e+01]
 [  5.95000000e+02   5.00000000e-01   5.15134722e+01]]


In [175]:
# 3100 cells by 3 dimensions (x,y,z)
xyz.shape
print xyz[0,:]
print xyz[1,:]

[  5.     0.5   40.09]
[  5.     0.5   40.07]


In [166]:
dat, map = transect_data.transect_data(["porosity",], keys=0, directory=mesh_dir, return_map=True)
keys, times, dataset = parse_ats.readATS(directory=mesh_dir)

IOError: Can't read data (File read failed: time = mon feb 19 17:49:27 2018
, filename = './test7/run21_19feb18_static_parameters\visdump_data.h5', file descriptor = 5, errno = 22, error message = 'invalid argument', buf = 000000000b1c3888, total read size = 46080, bytes this sub-read = 46080, bytes actually read = 18446744073709551615, offset = 1095456)

In [167]:
print dat.shape # (2 + # of varaiables, # of timesteps, NX, NZ)
NX = dat.shape[2]
NZ = dat.shape[3]
x = dat[0,0,:,:]
z = dat[1,0,:,:]

(3L, 1L, 60L, 96L)


In [157]:
# plot to see what is going on
fig, ax = plt.subplots(1,1)
transect_data.plot(dataset['porosity.cell.0']['0'][:][0,:], ax, directory=mesh_dir)
plt.show()

In [158]:
# depth coordinate
# z of the top surface elevation
z_surface = z[:,-1] + 0.006

depth = np.expand_dims(z_surface,1) - z
print "%8.16e"%z[0,-2]
print z[0,:]
print z_surface[0]
print depth[0,:]

4.0070000000000000e+01
[  4.11347  12.14042  21.08414  28.47951  32.17719  34.02603  34.95046
  35.51538  35.98615  36.37846  36.70538  36.97782  37.20485  37.39404
  37.5517   37.68308  37.79257  37.88381  37.95984  38.0232   38.076    38.12
  38.16     38.2      38.24     38.28     38.32     38.36     38.4      38.44
  38.48     38.52     38.56     38.6      38.64     38.68     38.72     38.76
  38.8      38.84     38.88     38.92     38.96     39.       39.04     39.08
  39.11     39.13     39.15     39.17     39.19     39.21     39.23     39.25
  39.27     39.29     39.31     39.33     39.35     39.37     39.39     39.41
  39.43     39.45     39.47     39.49     39.51     39.53     39.55     39.57
  39.59     39.61     39.63     39.65     39.67     39.69     39.71     39.73
  39.75     39.77     39.79     39.81     39.83     39.85     39.87     39.89
  39.91     39.93     39.95     39.97     39.99     40.01     40.03     40.05
  40.07     40.09   ]
40.096
[  3.59825300e+01   2.7955

In [159]:
print(abs(dat[2,0,50,:]))

[ 0.46901672  0.46622718  0.46333505  0.45929097  0.45742265  0.4568321
  0.4563397   0.45586843  0.45529679  0.45466464  0.45402201  0.45341074
  0.45285681  0.4523715   0.45195583  0.45160507  0.45131186  0.45106812
  0.45086612  0.45069896  0.45056066  0.45044624  0.45034289  0.45024028
  0.45013846  0.45003747  0.45        0.45        0.45        0.45        0.45
  0.45        0.45        0.45        0.45        0.45        0.45        0.45
  0.45        0.45        0.45        0.45        0.45        0.45        0.45
  0.45        0.45        0.45        0.45        0.45        0.45        0.45
  0.45        0.45        0.45        0.45        0.45        0.45        0.45
  0.45        0.45        0.45        0.45        0.45        0.45        0.45
  0.45        0.45        0.45        0.45        0.45        0.45        0.45
  0.45        0.45        0.45        0.45        0.45        0.45        0.45
  0.45        0.45        0.45        0.45        0.45        0.45        0.6

In [177]:
porosity = np.zeros((NX,NZ),'d')
porosity_mapped = np.zeros((NX*NZ,),'d')  # mapped data is a vector
permeability = np.zeros((NX,NZ),'d')
permeability_mapped = np.zeros((NX*NZ,),'d')  # mapped data is a vector
alpha = 1
for i in range(NX):
    for j in range(NZ):
        # ORGANIC REGION
        if abs(dat[2,0,i,j] - 0.65) < 1.e-6: 
            porosity[i,j] = np.multiply(0.2897,np.power(depth[i,j],-0.27))
            permeability[i,j] = np.multiply(5.e-14,np.power(depth[i,j],-2.43))
        # MINERAL REGION
        elif abs(dat[2,0,i,j] - 0.45) < 1.e-6:
            porosity[i,j] = 0.20
            permeability[i,j] = 1.e-14
        # DEEP MINERAL REGION
        elif abs(dat[2,0,i,j] - 0.45) > 1.e-6:
            porosity[i,j] = 0.40
            permeability[i,j] = 1.e-14
        else:
            raise RuntimeError("you screwed up your if statements")
            
print(porosity)
        
# map the data back into unstructured order
for i in range(NX):
    for j in range(NZ):
        porosity_mapped[map[i,j]] = porosity[i,j]
        permeability_mapped[map[i,j]] = permeability[i,j]
        

[[ 0.4         0.4         0.4        ...,  0.6652791   0.77607857
   1.15304872]
 [ 0.4         0.4         0.4        ...,  0.6652791   0.77607857
   1.15304872]
 [ 0.4         0.4         0.4        ...,  0.6652791   0.77607857
   1.15304872]
 ..., 
 [ 0.4         0.4         0.4        ...,  0.6652791   0.77607857
   1.15304872]
 [ 0.4         0.4         0.4        ...,  0.6652791   0.77607857
   1.15304872]
 [ 0.4         0.4         0.4        ...,  0.6652791   0.77607857
   1.15304872]]


In [179]:
# write this data into a file for use with ATS
#
# times : array of times [s] on which the data is defined
# ['base_porosity.cell.0'], ['permeability.cell.0'], ....  (key part is .cell.0, but prefer to use our names)
# within each data, ['0'], ['1'], ... ['LEN_TIMES-1'] : data groups
with h5py.File("soil_params_19Feb18.h5", 'w') as soil_h5:
    times = np.array([0.,],'d')
    soil_h5.create_dataset('time', data=times)
    poro = soil_h5.create_group("base_porosity.cell.0")
    poro.create_dataset('0', data=porosity_mapped)
    perm = soil_h5.create_group("permeability.cell.0")
    perm.create_dataset('0', data=permeability_mapped)

    
    

      <ParameterList name="base_porosity" type="ParameterList">
        <Parameter name="field evaluator type" type="string" value="independent variable from file" />
        <Parameter name="filename" type="string" value="soil_params.h5" />
        <Parameter name="variable name" type="string" value="base_porosity" />
        <Parameter name="constant in time" type="bool" value="true" />
      </ParameterList>